In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('/content/df_train_full.csv')
df_test = pd.read_csv('/content/df_test_clean.csv')

In [ ]:
print(df_train.shape)
print(df_test.shape)


(159829, 16)
(20199, 16)


In [ ]:
df_train.head()

,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
0,Alexithymia,my walgreens offbrand mucinex was engraved wit...,1.551641e+09,i.imgur.com,True,awxhir,https://external-preview.redd.it/WylDbZrnbvZdB...,NaN,2.0,12,mildlyinteresting,My Walgreens offbrand Mucinex was engraved wit...,0.84,1,0,0
1,prometheus1123,hackers leak emails from uae ambassador to us,1.496511e+09,aljazeera.com,True,6f2cy5,https://external-preview.redd.it/6fNhdbc6K1vFA...,NaN,1.0,44,neutralnews,Hackers leak emails from UAE ambassador to US,0.92,1,0,0
2,NaN,puppy taking in the view,1.471341e+09,i.imgur.com,True,4xypkv,https://external-preview.redd.it/HLtVNhTR6wtYt...,NaN,26.0,250,photoshopbattles,PsBattle: Puppy taking in the view,0.95,1,0,0
3,CrimsonBlue90,bride and groom exchange vows after fatal shoo...,1.423681e+09,independent.ie,True,2vkbtj,https://external-preview.redd.it/FQ-J9OIPFRpqi...,NaN,7.0,6,nottheonion,Bride and groom exchange vows after fatal shoo...,0.64,1,0,0
4,happenpupe,major thermos,1.495660e+09,i.redd.it,True,6d50rl,https://preview.redd.it/l9gvkkf3jizy.jpg?width...,NaN,0.0,2,pareidolia,major thermos,0.67,0,2,2


In [ ]:
df_train['hasImage'].value_counts()

,count
hasImage,
True,159829


In [ ]:
df_train_num = df_train.select_dtypes(include='number')


In [ ]:
df_train_num.corr()

,created_utc,num_comments,score,upvote_ratio,2_way_label,3_way_label,6_way_label
created_utc,1.000000,0.006224,0.010583,-0.051484,0.321150,-0.317252,-0.320232
num_comments,0.006224,1.000000,0.775130,0.064510,0.058606,-0.063502,-0.027569
score,0.010583,0.775130,1.000000,0.090273,0.033671,-0.032962,-0.024465
upvote_ratio,-0.051484,0.064510,0.090273,1.000000,-0.297227,0.274149,0.299626
2_way_label,0.321150,0.058606,0.033671,-0.297227,1.000000,-0.984148,-0.839593
3_way_label,-0.317252,-0.063502,-0.032962,0.274149,-0.984148,1.000000,0.751426
6_way_label,-0.320232,-0.027569,-0.024465,0.299626,-0.839593,0.751426,1.000000


In [ ]:
from transformers import RobertaTokenizer, RobertaModel
import torch
import numpy as np
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
model.to(device)
model.eval()

def extract_embeddings(df):
    texts = df['clean_title'].tolist()
    batch_size = 16
    features = []

    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        encoded = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**encoded)
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        features.append(cls_embeddings)

    return np.vstack(features)

X_train_bert = extract_embeddings(df_train)
X_test_bert = extract_embeddings(df_test)

Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1263/1263 [00:32<00:00, 38.72it/s]


In [ ]:
import numpy as np
import os
from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define the folder path
folder_path = '/content/drive/MyDrive/multimodel_dataset_extracted'

# 3. Define the full file paths
train_save_path = os.path.join(folder_path, 'X_text_emb_train.npy')
test_save_path = os.path.join(folder_path, 'X_text_emb_test.npy')

# 4. Save the NumPy arrays
# Assuming X_train_bert and X_test_bert are your embedding arrays
np.save(train_save_path, X_train_bert)
np.save(test_save_path, X_test_bert)

print(f"✅ Training embeddings saved successfully to: {train_save_path}")
print(f"✅ Testing embeddings saved successfully to: {test_save_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Training embeddings saved successfully to: /content/drive/MyDrive/multimodel_dataset_extracted/X_text_emb_train.npy
✅ Testing embeddings saved successfully to: /content/drive/MyDrive/multimodel_dataset_extracted/X_text_emb_test.npy


In [ ]:
cols_delete = ['clean_title','author','subreddit','domain', 'created_utc', 'hasImage', 'id', 'image_url', 'linked_submission_id', 'title']
df_train_mod = df_train.drop(columns=cols_delete)
df_test_mod = df_test.drop(columns=cols_delete)

In [ ]:
df_test_mod = df_test.drop(columns=cols_delete)
df_train_mod.head()

,num_comments,score,upvote_ratio,2_way_label,3_way_label,6_way_label
0,2.0,12,0.84,1,0,0
1,1.0,44,0.92,1,0,0
2,26.0,250,0.95,1,0,0
3,7.0,6,0.64,1,0,0
4,0.0,2,0.67,0,2,2


In [ ]:
df_test_mod.iloc[:,:3]

,num_comments,score,upvote_ratio
0,2.0,16,0.89
1,4.0,45,0.78
2,40.0,285,0.95
3,2.0,10,0.92
4,8.0,7,0.82
...,...,...,...
20194,0.0,19,0.92
20195,0.0,10,1.00
20196,8.0,6,0.81
20197,0.0,2,0.60


In [ ]:
from scipy.stats import skew

print("Skewness:", skew(df_train_mod['num_comments'].dropna()))


Skewness: 25.113121967419115


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. Apply log1p to reduce skewness
df_train_mod['num_comments_log'] = np.log1p(df_train_mod['num_comments'])

# 2. (Optional) Scale the log-transformed data
scaler = StandardScaler()
df_train_mod['num_comments_scaled'] = scaler.fit_transform(df_train_mod[['num_comments_log']])



In [ ]:
df_test_mod['num_comments_log'] = np.log1p(df_test_mod['num_comments'])
df_test_mod['num_comments_scaled'] = scaler.transform(df_test_mod[['num_comments_log']])

In [ ]:
from scipy.stats import skew

print("Skewness:", skew(df_train_mod['score'].dropna()))


Skewness: 12.201047332362547


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. Apply log1p to reduce skewness
df_train_mod['score_log'] = np.log1p(df_train_mod['score'])

# 2. (Optional) Scale the log-transformed data
scaler = StandardScaler()
df_train_mod['score_scaled'] = scaler.fit_transform(df_train_mod[['score_log']])



/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [ ]:
f_test_mod['score_log'] = np.log1p(df_test_mod['score'])

# 2. (Optional) Scale the log-transformed data
scaler = StandardScaler()
df_test_mod['num_comments_scaled'] = scaler.fit_transform(df_test_mod[['score_log']])

In [ ]:
df_test_mod['score_scaled'] = scaler.transform(df_test_mod[['score']])


In [ ]:
df_test_mod = df_test_mod.drop(columns=['score', 'num_comments'])
df_train_mod = df_train_mod.drop(columns=['score', 'num_comments'])

In [ ]:
df_train_mod.head()

,upvote_ratio,2_way_label,3_way_label,6_way_label,num_comments_scaled,score_scaled
0,0.84,1,0,0,-0.2,-0.117647
1,0.92,1,0,0,-0.4,0.823529
2,0.95,1,0,0,4.6,6.882353
3,0.64,1,0,0,0.8,-0.294118
4,0.67,0,2,2,-0.6,-0.411765
